In [1]:
import os
import glob

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

import sys
sys.path.append('..')

import modules.gait_metrics as gm
import modules.pose_estimation as pe


load_dir = os.path.join('..', 'data', 'kinect', 'best pos')
save_dir = os.path.join('..', 'data', 'results')

save_name = 'kinect_gait_metrics.csv'

# All files with .pkl extension
file_paths = glob.glob(os.path.join(load_dir, '*.pkl'))
save_path = os.path.join(save_dir, save_name)

df_metrics = pd.read_csv(save_path, index_col=0)


kinect_id = '2014-12-16_P005_Post_004'

matching_files = [f for f in file_paths if kinect_id in f]

for file_path in matching_files:

    df_head_feet = pd.read_pickle(file_path)
    
    # Convert all position vectors to float type 
    # so they can be easily input to linear algebra functions
    df_head_feet = df_head_feet.applymap(pd.to_numeric)

    # Cluster frames with k means to locate the 4 walking passes
    frames = df_head_feet.index.values.reshape(-1, 1)
    k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

    pass_dfs = gm.split_by_pass(df_head_feet,  k_means.labels_)
    
    df_gait_total = gm.combine_walking_passes(pass_dfs)
    
    


In [2]:
df_gait_total

,step_length_L,step_time_L,stride_width_L,stride_length_L,stride_time_L,stride_velocity_L,step_length_R,step_time_R,stride_width_R,stride_length_R,stride_time_R,stride_velocity_R
0,51.396655,0.433333,12.042193,108.501030,0.966667,112.242445,57.433271,0.533333,10.360671,119.855167,1.000000,119.855167
1,53.601627,0.433333,6.535507,109.475407,0.833333,131.370488,55.789605,0.400000,7.218874,115.287294,0.966667,119.262718
2,NaN,NaN,NaN,NaN,NaN,NaN,61.887649,0.533333,8.541389,121.472407,0.933333,130.149008
3,NaN,NaN,NaN,NaN,NaN,NaN,68.164883,0.600000,6.095472,124.408967,1.000000,124.408967
